In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-1x1x1/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 5966.29it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        # if i_q == epsilon.qpts.index_q0:
        #     key = gkspc.g_norm2
        # else:
        #     key = gkspc.gk_norm2

        key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 372.17it/s]

Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)


In [7]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [8]:
mvc = kernelclass.charge_mtxel.mvc(0)
mvvp = kernelclass.charge_mtxel.mvvp(0, 0)
mccp = kernelclass.charge_mtxel.mccp(0, 0)

mvc = mvc[0, 0, :]
mvvp = mvvp[0, 0, :]
mccp = mccp[0, 0, :]

In [9]:
# # print mvc in rows
# for i in range(mvc.shape[0]):
#     print(f"Row {i+1}: {mvc[i]}")

In [10]:
# # print mvvp in rows
# for i in range(mvvp.shape[0]):
#     print(f"Row {i+1}: {mvvp[i]}")

In [11]:
# # print mccp in rows
# for i in range(mccp.shape[0]):
#     print(f"Row {i+1}: {mccp[i]}")

In [12]:
data = kernelclass.calc_kernel_mtxel()

In [13]:
exc = data["exc"]
print(f"The exc kernel is {exc}")

head = data["head"]
print(f"The head kernel is {head}")

wings = data["wings"]
print(f"The wings kernel is {wings}")

body = data["body"]
print(f"The body kernel is {body}")

The exc kernel is [[[[[[0.39326425+2.77555756e-17j]]]]]]
The head kernel is [[[[[[1.-1.89016773e-19j]]]]]]
The wings kernel is [[[[[[0.+0.j]]]]]]
The body kernel is [[[[[[-0.020828-9.12308222e-09j]]]]]]


Calculate $W_{G, G'}(q, 0)$ 

In [14]:
norm_array = kernelclass.l_gq[0].g_norm2
vq0g = np.where(norm_array == 0, 0, 1 / np.where(norm_array == 0, 1, norm_array))
sort_order_0 = sort_cryst_like_BGW(
    kernelclass.l_gq[0].g_cryst, norm_array
)
vq0g = vq0g[sort_order_0]

# for i, value in enumerate(vq0g):
#     print(f"Row {i+1}: {value}")

In [15]:
i_q = 0
epsinv = kernelclass.l_epsmats[i_q]
epsinv = np.conjugate(epsinv)
    
sort_order = sort_cryst_like_BGW(
    kernelclass.l_gq_epsinv[i_q].gk_cryst, kernelclass.l_gq_epsinv[i_q].gk_norm2
)
            
# As given in the sigma code, I need to use another sorting.
sort_order_QTM = np.argsort(sort_order)

# But my matrices, vcoul, etc. are sorted according to BGW.
sort_BGW = sort_cryst_like_BGW(
    kernelclass.l_gq_epsinv[i_q].g_cryst, kernelclass.l_gq_epsinv[i_q].g_norm2
)

sort_order_BGW = sort_order_QTM[sort_BGW]

epsiloninv = (
    reorder_2d_matrix_sorted_gvecs(epsinv, sort_order_BGW)
)

# # Print epsinv in rows and columns
# # Exclude the first row and column
# epsiloninv_excluded = epsiloninv[1:, 1:]

# # Print the modified epsinv in rows and columns
# for row_idx, row in enumerate(epsiloninv_excluded):
#     for col_idx, value in enumerate(row):
        
#         print(f"Row {row_idx + 2}, Column {col_idx + 2}: {value}")

In [16]:
wcoul = np.zeros((len(mccp), len(mccp)), dtype=complex)
wcoul[0, 0] = 1
wcoul[0, 1:] = 0
wcoul[1:, 0] = 0

for i in range(1, len(epsiloninv)):
    for j in range(1, len(epsiloninv)):
        wcoul[i, j] = epsiloninv[j, i] * vq0g[j]

for i in range(len(epsiloninv), len(wcoul)):
    wcoul[i, i] = vq0g[i]

# Exclude the first row and column
wcoul_excluded = wcoul[1:15, 1:15]
# Print the modified wcoul in rows and columns
for row_idx, row in enumerate(wcoul_excluded):
    for col_idx, value in enumerate(row):
        print(f"Row {row_idx + 2}, Column {col_idx + 2}: {value}")

Row 2, Column 2: (0.7231673221062815+5.952640332278535e-21j)
Row 2, Column 3: (-0.00938159571519793+1.9243440210863934e-06j)
Row 2, Column 4: (-0.009388520143503689+2.7461936405165256e-06j)
Row 2, Column 5: (-0.009393546608738049-1.5928194270633177e-06j)
Row 2, Column 6: (0.009574931881070912+2.1666085261531803e-07j)
Row 2, Column 7: (0.009571409418301596-4.9322776734594835e-08j)
Row 2, Column 8: (0.009563225678594705+1.5829620818967005e-06j)
Row 2, Column 9: (0.15531890453231015+1.199961795676428e-06j)
Row 2, Column 10: (-0.03600521229523887-9.766019794058247e-07j)
Row 2, Column 11: (-0.036005327138782565+2.2465886033250723e-06j)
Row 2, Column 12: (-0.0360055912282337+1.1521709666044322e-06j)
Row 2, Column 13: (0.03611918226109624+2.8220523925712743e-06j)
Row 2, Column 14: (0.03611884586119633-4.831680153487085e-07j)
Row 2, Column 15: (0.03611881479449673-1.861305826011002e-07j)
Row 3, Column 2: (-0.009369078588677105-1.921776519958363e-06j)
Row 3, Column 3: (0.692245539839088+9.30100

In [17]:
for row in range(wcoul.shape[0]):
    row_data = []
    for col in range(wcoul.shape[1]):
        row_data.append(f"{wcoul[row, col]:.7g}")
    print(" | ".join(row_data))

1+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j
0+0j | 0.7231673+5.95264e-21j | -0.009381596+1.924344e-06j | -0.00938852+2.746194e-06j | -0.009393547-1.592819e-06j | 0.009574932+2.166609e-07j | 0.009571409-4.932278e-08j | 0.009563226+1.582962e-06j | 0.1553189+1.199962e-06j | -0.03600521-9.76602e-07j | -0.03600533+2.246589e-06j | -0.03600559+1.152171e-06j | 0.03611918+2.822052e-06j | 0.03611885-4.83168e-07j | 0.03611881-1.861306e-07j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j | 0+0j
0+0j | -0.009369079-1.921777e-06j | 0.6922455+9.301001e-23j | 0.006061328-4.626786e-07j | 0.00606583+2.320433e-07j | -0.00611522-1.034634e-06j | -0.006111204+2.313775e-06j | 0.1860175+1.906234e-07j | 0.009246807-3.380863e-07j | -0.0360641-3.863236e-07j | -0.03606477+6.485488e-07j | -0.03643515+9.430195e-07j | 0.03560726-6.43541e-07j | 0.0

In [18]:
for row in range(mvvp.shape[0]):
    row_data = []
    row_data.append(f"{mvvp[row]:.7g}")
    print(" | ".join(row_data))

1-2.713627e-19j
0.2551328+1.364681e-07j
0.4264408-3.906275e-06j
0.4258757-9.362311e-07j
0.4234961-1.729757e-06j
0.4234961+1.729757e-06j
0.4258757+9.362311e-07j
0.4264408+3.906275e-06j
0.2551328-1.364681e-07j
0.2976161-1.530232e-06j
0.2955457-1.174972e-06j
0.2950541-2.494651e-06j
0.2950541+2.494651e-06j
0.2955457+1.174972e-06j
0.2976161+1.530232e-06j
0.2721664-3.77815e-08j
0.2729211-1.096804e-07j
0.2760775-1.534221e-07j
0.1545191-4.363413e-07j
0.1543571-6.476026e-07j
0.1545626+1.916415e-07j
0.1545626-1.916415e-07j
0.1543571+6.476026e-07j
0.1545191+4.363413e-07j
0.2760775+1.534221e-07j
0.2729211+1.096804e-07j
0.2721664+3.77815e-08j


In [19]:
wcoul_body = wcoul[1:, 1:]
mvvp_body = mvvp[1:]
mccp_body = mccp[1:]

bodyval = 0
for i in range(wcoul_body.shape[0]):
    for j in range(wcoul_body.shape[1]):
        bodyval += mvvp_body[i] * np.conjugate(wcoul_body[j, i]) * (mccp_body[j])
print(f"The body kernel is {bodyval}")

The body kernel is (-0.02083082965554543-2.0935194876814875e-08j)


In [20]:
from kernelv2 import KernelMtxElv2


kernelclassv2 = KernelMtxElv2.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [21]:
datav2 = kernelclassv2.kernel_mtxel()

excv2 = datav2["exc"]
print(f"The exc kernel is {excv2}")

headv2 = datav2["head"]
print(f"The head kernel is {headv2}")

wingsv2 = datav2["wings"]
print(f"The wings kernel is {wingsv2}")

bodyv2 = datav2["body"]
print(f"The body kernel is {bodyv2}")

The exc kernel is [[[[[[0.39326425+2.77555756e-17j]]]]]]
The head kernel is [[[[[[1.-1.89016773e-19j]]]]]]
The wings kernel is [[[[[[0.+0.j]]]]]]
The body kernel is [[[[[[-0.02083083+1.04335341e-08j]]]]]]
